# Random Forest


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from dataframe_builder import DataFrameBuilder

builder = DataFrameBuilder()
df = builder.build()

Buscando dataframe en /home/marat/projects/tp-aa/cache/dataframe.pickle
Encontrado. Dimensiones: (80995, 56)


Saco la columna de text porque ocupa mucho espacio

# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [2]:
from sklearn.ensemble import RandomForestClassifier
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble.forest:

class RandomForestClassifier(ForestClassifier)
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and use averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |      Note: this parameter is tree-specific.
 |  
 |  max_features : int, fl



Opciones:

- n_estimators: cantidad de árboles
- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [3]:
from search import find_best_classifier

options = {
    'n_estimators': range(10, 100, 5),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}



classifier = find_best_classifier(
    df, RandomForestClassifier, param_distributions=options, 
    scoring='roc_auc', cv=10, n_jobs=6, n_iter=0)    

Buscando parámetros para RandomForestClassifier


KeyboardInterrupt: 

In [4]:
from helpers import get_scores

get_scores(classifier)

NameError: name 'classifier' is not defined